<a href="https://colab.research.google.com/github/annabm99/DiseaseTraitPleiotropy/blob/main/Func_analysis_BMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Preparation

##Install package(s)

In [1]:
!pip install easy_entrez
!pip install pylatex
!apt-get install -y texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pylatex: filename=PyLaTeX-1.4.2-py3-none-any.whl size=43560 sha256=0a73b64b49395961f4b9ce0cf3d11c94700d32ee83c2b8373fe5128d6df77020
  Stored in directory: /root/.cache/pip/wheels/a3/60/09/c6f9f98feac18da1b5fc02bd765c6b3cb9a0f75955a12b27ad
Successfully built pylatex
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pandoc is already the newest version (2.9.2.1-3ubuntu2).
pandoc set to manually installed.
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 libapache-pom-java libcommons-logging-java libcommons-parent-java
  libfontbox-java libfontenc1 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 li

## Libraries

In [2]:
import pandas as pd
import numpy as np

## Import files

In [3]:
# Manual import of text files
CAD_p = 'CAD_d_vs_BMI_t-positiveIDs.txt'
CAD_n = 'CAD_d_vs_BMI_t-negativeIDs.txt'
HT_p = 'HT_d_vs_BMI_t-positiveIDs.txt'
HT_n = 'HT_d_vs_BMI_t-negativeIDs.txt'
STR_p = 'STR_d_vs_BMI_t-positiveIDs.txt'
STR_n = 'STR_d_vs_BMI_t-negativeIDs.txt'
T2D_p = 'T2D_d_vs_BMI_t-positiveIDs.txt'
T2D_n = 'T2D_d_vs_BMI_t-negativeIDs.txt'

# Read the files and store their contents in a dictionary, keys are the names of the dataframes, values are the dataframes per se.
data = {}
for name, df in {'CAD_p': CAD_p, 'CAD_n': CAD_n, 'HT_p': HT_p, 'HT_n': HT_n, 'STR_p': STR_p, 'STR_n': STR_n, 'T2D_p': T2D_p, 'T2D_n': T2D_n}.items():
    with open(df, 'r') as file:
        data[name] = file.read()

The files containing rsIDs are stored in the data dictionary.

#2. Gene search

## Packages
The easy_entrez package allows connecting to the NCBI server through API and retrieve information.

In [4]:
from easy_entrez import EntrezAPI

##Functions

Get gene names from the rsID list:

In [5]:
def GetGene(lst):
    try:
        # Create an empty list to store rsID-gene pairs
        rsid_gene_pairs = []

        # Iterate through rsIDs (each is one line)
        for rsID in lst.splitlines():
            # Fetch data from Entrez
            mask = entrez_api.fetch([rsID], max_results=3, database='snp').data

            # Check if gene association is found
            if isinstance(mask, dict):
                continue  # Skip to the next rsID if no gene association is found

            # Extract gene names
            namespaces = {'ns0': 'https://www.ncbi.nlm.nih.gov/SNP/docsum'}
            genes = [
                name.text
                for name in mask.findall('.//ns0:GENE_E/ns0:NAME', namespaces)
            ]

            # If gene association is found, append to list
            if genes:
                for gene in genes:
                    rsid_gene_pairs.append({'rsID': rsID, 'Gene': gene})

        # Convert list of dictionaries to DataFrame
        rsid_gene_df = pd.DataFrame(rsid_gene_pairs)

        return rsid_gene_df

    # Error catching
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None


Group all rsIDs by gene:

In [6]:
def GroupByGene(rsid_gene_df):
    try:
        # Initialize an empty list to store rows for the result DataFrame
        result_rows = []

        # Initialize an empty dictionary to store gene-rsID associations
        gene_rsID_dict = {}

        # Iterate over each row in the DataFrame
        for index, row in rsid_gene_df.iterrows():
            rsID = row['rsID']
            gene = row['Gene']

            # If gene is not already a key in the dictionary, add it with an empty list as value
            if gene not in gene_rsID_dict:
                gene_rsID_dict[gene] = []

            # Append rsID to the list associated with the gene
            gene_rsID_dict[gene].append(rsID)

        # Iterate over the gene-rsID dictionary and create rows for the result DataFrame
        for gene, rsIDs in gene_rsID_dict.items():
            rsid_str = ', '.join(rsIDs)
            result_rows.append({'Gene': gene, 'rsIDs': rsid_str})

        # Create the result DataFrame
        result_df = pd.DataFrame(result_rows)

        return result_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

## Run gene search

Connect to the server:

In [7]:
from easy_entrez import EntrezAPI

entrez_api = EntrezAPI('Pleiotropies', 'anna.basquet01@estudiant.upf.edu')

Run both functions and create a new results dictionary with the resulting information.

In [37]:
# Apply the functions to each DataFrame and store the results with modified names into a dictionary
results = {}

# Iterate through all dataframes stored in data
for name, df in data.items():
    # Modify the key name to include the suffix
    name1 = f"{name}_genes"

    # Apply the first function to the initial dataframe and store the result
    gene_df = GetGene(df)

    if gene_df is not None:
        results[name1] = gene_df
        print(f"Gene extraction completed for {name}. Dataframe stored.")
    else:
        print(f"Gene extraction FAILED for {name}. Skipping to the next dataframe.")
        continue

    # Apply the second function to the previously generated dataframe and store the result
    name2 = f"{name}_table"
    gene_table = GroupByGene(gene_df)

    if gene_table is not None:
        results[name2] = gene_table
        print(f"Table groupping by gene completed for {name}. Dataframe stored.")
    else:
        print(f"Gene groupping FAILED for {name}. Skipping to the next dataframe.")
        continue

Gene extraction completed for CAD_p. Dataframe stored.
Table groupping by gene completed for CAD_p. Dataframe stored.
Gene extraction completed for CAD_n. Dataframe stored.
Table groupping by gene completed for CAD_n. Dataframe stored.
Gene extraction completed for HT_p. Dataframe stored.
Table groupping by gene completed for HT_p. Dataframe stored.
Gene extraction completed for HT_n. Dataframe stored.
Table groupping by gene completed for HT_n. Dataframe stored.
Gene extraction completed for STR_p. Dataframe stored.
Table groupping by gene completed for STR_p. Dataframe stored.
Gene extraction completed for STR_n. Dataframe stored.
Table groupping by gene completed for STR_n. Dataframe stored.
Gene extraction completed for T2D_p. Dataframe stored.
Table groupping by gene completed for T2D_p. Dataframe stored.
Gene extraction completed for T2D_n. Dataframe stored.
Table groupping by gene completed for T2D_n. Dataframe stored.


#3. Overlapping genes

In [34]:
def FindOverlapping(df1, df2, name1, name2):

  # Change rsIDs column names to differentiate between diseases
  df1 = df1.rename(columns={'rsIDs': f'rsIDs_{name1}'})
  df2 = df2.rename(columns={'rsIDs': f'rsIDs_{name2}'})

  # Merge the dataframes by the gene name
  merged_df = pd.merge(df1, df2, on='Gene')

  # Create new empty column for overlapping rsIDs
  merged_df['overlap_rsIDs']=np.nan

  # Find the overlapping rsIDs
  for index, row in merged_df.iterrows():
    # Grab the rsIDs of each disease
    rsIDs1 = set(rsID.strip() for rsID in row[f'rsIDs_{name1}'].split(','))
    rsIDs2 = set(rsID.strip() for rsID in row[f'rsIDs_{name2}'].split(','))

    # Find the intersection between the sets
    overlap_rsIDs = rsIDs1.intersection(rsIDs2)

    # Put them in a list and add them to the dataframe
    merged_df.at[index, 'overlap_rsIDs'] = ','.join(overlap_rsIDs)

  # Check if merged_df is empty
  if merged_df.empty:
    return None # Return nothing if empty
  else:
    return merged_df # Return dataframe

In [38]:
# Initialize objects for the loop
overlapping_dfs = list()
compared_pairs = set()

for name1, df1 in ((name, df) for name, df in results.items() if name.endswith('_table')):
    for name2, df2 in ((name, df) for name, df in results.items() if name.endswith('_table')):

      # Ensure we are comparing different dataframes and not repeating
      if name1 != name2 and (name1, name2) not in compared_pairs and (name2, name1) not in compared_pairs:
            # Grab prefix of each dataframe
            pref1 = '_'.join(name1.split('_')[:2])
            pref2 = '_'.join(name2.split('_')[:2])

            # Find overlapping genes between the two dataframes
            ovl_name = f'{pref1}_{pref2}'
            ovl_df = FindOverlapping(df1, df2, pref1, pref2)

            if ovl_df is not None:
                overlapping_dfs.append([ovl_name, ovl_df])
                print(f"Overlapping genes found for {pref1} and {pref2}. Dataframe stored.")
            else:
                print(f"No overlapping genes found between {pref1} and {pref2}. Skipping to the next dataframe.")

            # Add the pair to the set of compared pairs
            compared_pairs.add((name1, name2))

# Update the results dictionary after the iteration is complete
for i, df_list in enumerate(overlapping_dfs):
    name, df = df_list
    results[f'{name}_overlap'] = df

No overlapping genes found between CAD_p and CAD_n. Skipping to the next dataframe.
Overlapping genes found for CAD_p and HT_p. Dataframe stored.
No overlapping genes found between CAD_p and HT_n. Skipping to the next dataframe.
No overlapping genes found between CAD_p and STR_p. Skipping to the next dataframe.
No overlapping genes found between CAD_p and STR_n. Skipping to the next dataframe.
Overlapping genes found for CAD_p and T2D_p. Dataframe stored.
No overlapping genes found between CAD_p and T2D_n. Skipping to the next dataframe.
No overlapping genes found between CAD_n and HT_p. Skipping to the next dataframe.
Overlapping genes found for CAD_n and HT_n. Dataframe stored.
No overlapping genes found between CAD_n and STR_p. Skipping to the next dataframe.
Overlapping genes found for CAD_n and STR_n. Dataframe stored.
No overlapping genes found between CAD_n and T2D_p. Skipping to the next dataframe.
Overlapping genes found for CAD_n and T2D_n. Dataframe stored.
No overlapping ge

# 5. Create output file

In [48]:
from pylatex import Document, Tabular, NoEscape, Package, Section, Subsection, MediumText
from pylatex.utils import bold

In [45]:
def ConstructTitlePleiotropy(name):
    options = {
        'CAD': 'Coronary Artery Disease',
        'STR': 'Stroke',
        'T2D': 'Type 2 Diabetes',
        'HT': 'Hypertension',
        'p': 'synergistic (positive)',
        'n': 'antagonistic (negative)'
    }
    direction = ''
    disease = ''
    for part in name.split('_'):
        if part in options:
            if part in ['p', 'n']:
                direction = options[part]
            else:
                disease = options[part]
    return f"Genes involved in {direction} pleiotropies between {disease} and Body Mass Index."

def ConstructTitleOverlap(name):
    options = {
        'CAD': 'Coronary Artery Disease',
        'STR': 'Stroke',
        'T2D': 'Type 2 Diabetes',
        'HT': 'Hypertension',
        'p': 'synergistic (positive)',
        'n': 'antagonistic (negative)'
    }
    direction1 = ''
    direction2 = ''
    disease1 = ''
    disease2 = ''
    for part1 in name.split('_')[:2]:
      if part1 in ['p', 'n']:
        direction1 = options[part1]
      else:
        disease1 = options[part1]
    for part2 in name.split('_')[2:4]:
      if part2 in ['p', 'n']:
        direction2 = options[part2]
      else:
        disease2 = options[part2]


    return f"Overlapping genes involved in {direction1} pleiotropies for {disease1} and {direction2} pleiotropies for {disease2} with Body Mass Index."

In [50]:
# Create a new document
doc = Document()

# Add the geometry package with custom margin settings
doc.packages.append(Package('geometry', options=['margin=1.5cm']))

# Add a section
section_genes = Section('Pleiotropic genes')

for name, df in results.items():
    if name.endswith('_table'):

        subsection = Subsection(ConstructTitlePleiotropy(name))

        table = Tabular('l p{15cm}', row_height='0.9', booktabs=True)
        table.add_row(bold('Gene'), bold('Overlapping rsIDs'))
        table.add_hline()  # Add a horizontal line

        # Add rows to the table
        for index, row in df.iterrows():
            # Use NoEscape to prevent pylatex from escaping special LaTeX characters
            table.add_row(NoEscape(row['Gene']), NoEscape(row['rsIDs']))
            table.add_hline(color='lightgray')  # Add a horizontal line

        # Add the table to the subsection
        subsection.append(NoEscape('\\begin{center}'))
        subsection.append(table)
        subsection.append(NoEscape('\\end{center}'))

        # Add the subsection to the general section
        section_genes.append(subsection)

# Add section to the document after it has been constructed
doc.append(section_genes)

section_overlap = Section('Overlapping genes')

for name, df in results.items():
    if name.endswith('_overlap'):

        subsection = Subsection(ConstructTitleOverlap(name))
        # Create a new tabular environment with custom column format
        table = Tabular('l p{15cm}', row_height='0.9', booktabs=True)
        table.add_row(bold('Gene'), bold('rsIDs'))
        table.add_hline()  # Add a horizontal line

        # Add rows to the table
        for index, row in df.iterrows():
            # Use NoEscape to prevent pylatex from escaping special LaTeX characters
            table.add_row(NoEscape(row['Gene']), NoEscape(row['overlap_rsIDs']))
            table.add_hline(color='lightgray')  # Add a horizontal line

        # Add the table to the document
        subsection.append(NoEscape('\\begin{center}'))
        subsection.append(table)
        subsection.append(NoEscape('\\end{center}'))

        # Add subsection to general section
        section_overlap.append(subsection)

# Add section to the document after it has been constructed
doc.append(section_overlap)

# Save the document to a file and compile it to a PDF
doc.generate_pdf('BMI_genes', clean_tex=False)